# Feast

An open source feature store for machine learning

In [19]:
from feast import FeatureStore

# You may need to update the path depending on where you stored it
feast_repo_path = "feast_repo/feature_repo/"
store = FeatureStore(repo_path=feast_repo_path)

In [20]:
# Inject some data into the feature store

import pandas as pd
from datetime import datetime

entity_df = pd.DataFrame.from_dict({
    "driver_id": [1001, 1002, 1003, 1004],
    "event_timestamp": [
        datetime(2021, 4, 12, 10, 59, 42),
        datetime(2021, 4, 12, 8,  12, 10),
        datetime(2021, 4, 12, 16, 40, 26),
        datetime(2021, 4, 12, 15, 1 , 12)
    ]
})

training_df = store.get_historical_features(
    entity_df=entity_df,
    features = [
        'driver_hourly_stats:conv_rate',
        'driver_hourly_stats:acc_rate',
        'driver_hourly_stats:avg_daily_trips'
    ],
).to_df()

print(training_df.head())

!cd feast_repo/feature_repo && feast materialize-incremental $(date -u +"%Y-%m-%dT%H:%M:%S")

   driver_id           event_timestamp  conv_rate  acc_rate  avg_daily_trips
0       1001 2021-04-12 10:59:42+00:00   0.270642  0.683712              751
1       1002 2021-04-12 08:12:10+00:00   0.918094  0.040282              919
2       1003 2021-04-12 16:40:26+00:00   0.702248  0.338838              333
3       1004 2021-04-12 15:01:12+00:00   0.560620  0.903400              858
Materializing 2 feature views to 2023-10-04 21:30:24+02:00 into the sqlite online store.

driver_hourly_stats from 2023-10-04 21:30:20+02:00 to 2023-10-04 21:30:24+02:00:
0it [00:00, ?it/s]
driver_hourly_stats_fresh from 2023-10-04 21:30:20+02:00 to 2023-10-04 21:30:24+02:00:
0it [00:00, ?it/s]


Here we will set up a custom FeastPromptTemplate. This prompt template will take in a driver id, look up their stats, and format those stats into a prompt.

In [21]:
from langchain.prompts import PromptTemplate, StringPromptTemplate

template = """Given the driver's up to date stats, write them note relaying those stats to them.
If they have a conversation rate above .5, give them a compliment. Otherwise, make a silly joke about chickens at the end to make them feel better

Here are the drivers stats:
Conversation rate: {conv_rate}
Acceptance rate: {acc_rate}
Average Daily Trips: {avg_daily_trips}

Your response:"""
prompt = PromptTemplate.from_template(template)

In [22]:
class FeastPromptTemplate(StringPromptTemplate):
    def format(self, **kwargs) -> str:
        driver_id = kwargs.pop("driver_id")
        feature_vector = store.get_online_features(
            features=[
                "driver_hourly_stats:conv_rate",
                "driver_hourly_stats:acc_rate",
                "driver_hourly_stats:avg_daily_trips",
            ],
            entity_rows=[{"driver_id": driver_id}],
        ).to_dict()
        kwargs["conv_rate"] = feature_vector["conv_rate"][0]
        kwargs["acc_rate"] = feature_vector["acc_rate"][0]
        kwargs["avg_daily_trips"] = feature_vector["avg_daily_trips"][0]
        return prompt.format(**kwargs)

In [23]:
prompt_template = FeastPromptTemplate(input_variables=["driver_id"])

In [24]:
print(prompt_template.format(driver_id=1001))

Given the driver's up to date stats, write them note relaying those stats to them.
If they have a conversation rate above .5, give them a compliment. Otherwise, make a silly joke about chickens at the end to make them feel better

Here are the drivers stats:
Conversation rate: 0.9924231171607971
Acceptance rate: 0.6750268340110779
Average Daily Trips: 556

Your response:


### Use in a chain

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

chain = LLMChain(llm=ChatOpenAI(), prompt=prompt_template)

chain.run(1001)

"Hi there! I hope you're doing well. I wanted to update you on your current stats as a driver. Your conversation rate is an impressive 0.9924, which means you're doing an outstanding job engaging with passengers. Keep up the fantastic work!\n\nAdditionally, your acceptance rate stands at 0.6750, indicating that you're making good choices when it comes to accepting trips. Way to go!\n\nLastly, your average daily trips are at a solid 556. That's quite a number and shows your dedication and hard work.\n\nKeep up the great work, and remember, every trip counts! If you ever feel down about your conversation rate, just remember this silly joke: Why did the chicken go to the seance? To talk to the other side! Stay positive and keep shining as a driver!"